<a href="https://colab.research.google.com/github/dercodeKoenig/colab-KI/blob/main/text_ki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KI entscheidet ob ein Text richtig oder eine sinnlose Zeichenkombination ist<br>


1: Trainingstext laden

In [ ]:
!wget https://raw.githubusercontent.com/dercodeKoenig/colab-KI/main/in.txt       #trainingstext

2: Imports ausführen

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

3: Textdatei lesen und verschiedenen Buchstaben eine Id zuweisen


In [ ]:
raw_text=str(open("in.txt").read()) #alle zeichen hier rein
raw_text2="QWERTZUIOPÜ*'ASDFGHJKLÖÄYXCVBNM;:_qwertzuiopü+#asdfghjklöäyxcvbnm,.-<>|@1234567890ß!\"§$%&/()=? \n»«~+*\\—½{}[]è" #alle zeichen hier rein
chars = sorted(list(set(raw_text2)))
print(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_vocab = len(chars)
print ("Total Vocab: ", n_vocab)
n_chars = len(raw_text)
print ("Total Characters: ", n_chars)

4: Text in kleine Ausschnitte zerhacken mit der Länge seq_length und Eingabe-Ausgabe-Paare generieren. <br>
Um dem Modell auch falschen Text zu zeigen wird für jeden wahren String auch ein Falscher mit zufälligen Zeichen aus chars erstellt. <br>So bekommt das Modell die gleiche Zahl an richtigem und falschem Text zum üben

In [ ]:
seq_length = 5
dataX = []
dataY = []
for i in range(n_chars-seq_length):
    data=raw_text[i:i+seq_length];
    random_string = ''
    for _ in range(seq_length):
        random_integer = random.randint(0,n_vocab-1)
        random_string += (int_to_char[random_integer])
        
    dataX+=data
    dataX+=random_string
    dataY+=[1]
    dataY+=[0]

dataX=np.reshape(np.array(dataX),(int(len(dataX)/seq_length),seq_length)) #reshape in [Anzahl, Länge einer Sequenz]
Y=np.array(dataY)

print("X:")
print(dataX)
print("")
print("Y:")
print(Y)

5: Jeden Buchstaben in seine Id umwandeln

In [ ]:

X = []
for i in range(len(dataX)):
  for o in range(len(dataX[i])):
    X+=[char_to_int[dataX[i][o]]]

X = np.reshape(X, (int(len(X)/seq_length), seq_length))
print(X)

6: One-hot-encoding durchführen.
Beim one-hot-encoding werden Werte aus vielen 0 und einer 1 dargestellt.<br>
Zb ["a","b","c"] wird zu [[0,0,1],[0,1,0],[1,0,0]]

In [ ]:
X = to_categorical(X,num_classes=n_vocab)
# reshape in [Anzahl, Länge der Sequenz, Länge der one-hot-encode-vectoren]
X = np.reshape(X, (int(len(X)), seq_length, n_vocab))

7: Modell erstellen. Testweise kann als LSTM auch GRU verwendet werden

In [ ]:
filepath="weights.hdf5"

model = Sequential()
model.add(LSTM(32, input_shape=(seq_length, n_vocab),return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
try:
  model.load_weights(filepath)
except:
    pass

print(model.summary())

8: Modell trainieren

In [ ]:
# checkpoint festlegen
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, Y, epochs=10, batch_size=128,verbose=1, callbacks=callbacks_list)

9: Auswertung

In [ ]:
while True:
  eingabe = input("text: ")
  if(eingabe=="ende"):
    break
  n_chars = len(eingabe)
  print ("Buchatsben: ", n_chars)
  dataX = []
  for i in range(n_chars-seq_length):
    data=eingabe[i:i+seq_length];
    dataX+=data
  dataX=np.reshape(np.array(dataX),(int(len(dataX)/seq_length),seq_length))
  Xeingabe = []
  for i in range(len(dataX)):
    for o in range(len(dataX[i])):
      Xeingabe+=[char_to_int[dataX[i][o]]]
  Xeingabe = to_categorical(Xeingabe,num_classes=n_vocab)

  # reshape X to be [samples, time steps, features]
  Xeingabe = np.reshape(Xeingabe, (int(len(Xeingabe)/seq_length), seq_length, n_vocab))
  wahr = 0.0
  falsch = 0.0
  gesamt = 0
  for i in Xeingabe:
    gesamt+=1
    Xeingabe=np.reshape(np.array(i),(1,seq_length,n_vocab))
    res = model.predict(Xeingabe)
    
    wahr += res[0][1]
    falsch +=res[0][0]
  
  print("Bewertung Text: ",((wahr - falsch)/gesamt))
  print("")